In [1]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

con = sqlite3.connect('../summer')
cur = con.cursor()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
recordings_test = ['\'week7_3\'', '\'week8_3\'']
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']

all_weeks_test = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings_test)), con)

In [3]:
playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

motion_output = []
def create_labels(row):
    output = np.zeros(11)
    output[playerIdx[row['PlayerName']]] = 1
    motion_output.append(output)
    
all_weeks_test.apply(lambda x: create_labels(x), axis=1)
testing_y = np.asarray(motion_output)

all_weeks_test = all_weeks_test.drop(columns=['PlayerName'])
testing_x = np.asarray(all_weeks_test)

print(testing_x.shape)
print(testing_y.shape)

(919446, 21)
(919446, 11)


In [5]:
tf.reset_default_graph()

x = tf.placeholder("float", [None, 21])
y = tf.placeholder("float", [None, 11])

with tf.Session() as sess:
    model = tf.train.import_meta_graph('models_basic_nn/19_19_02_31/basic_nn_19_19_02_31.meta')
    model.restore(sess, tf.train.latest_checkpoint('models_basic_nn/19_19_02_31'))
    graph = tf.get_default_graph()
        
    W1 = sess.run('W1:0')
    W2 = sess.run('W2:0')
    #np.savetxt('models_basic_nn/19_19_02_31/W1.csv', W1)
    #np.savetxt('models_basic_nn/19_19_02_31/W2.csv', W2)
    
    b1 = sess.run('b1:0')
    b2 = sess.run('b2:0')
    #np.savetxt('models_basic_nn/19_19_02_31/b1.csv', b1)
    #np.savetxt('models_basic_nn/19_19_02_31/b2.csv', b2)
        
    layer_1 = tf.nn.sigmoid(tf.matmul(testing_x, W1) + b1)
    layer_2 = tf.nn.softmax(tf.matmul(layer_1, W2) + b2)
    
    print(layer_2.shape)
    
    predictions = tf.equal(tf.argmax(layer_2, 1), tf.argmax(testing_y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions,  tf.float64))
    print("Testing accuracy:", accuracy.eval({x: testing_x, y: testing_y}))

INFO:tensorflow:Restoring parameters from models_basic_nn/19_19_02_31/basic_nn_19_19_02_31
(919446, 11)
Testing accuracy: 0.31811221104882725


In [6]:
random_headpos_x = all_weeks_test.copy(deep=True)
random_headpos_x['HeadPos_x'] = random_headpos_x['HeadPos_x'].sample(frac=1).values
random_headpos_x = np.asarray(random_headpos_x)
random_headpos_y = all_weeks_test.copy(deep=True)
random_headpos_y['HeadPos_y'] = random_headpos_y['HeadPos_y'].sample(frac=1).values
random_headpos_y = np.asarray(random_headpos_y)
random_headpos_z = all_weeks_test.copy(deep=True)
random_headpos_z['HeadPos_z'] = random_headpos_z['HeadPos_z'].sample(frac=1).values
random_headpos_z = np.asarray(random_headpos_z)

random_lefthand_x = all_weeks_test.copy(deep=True)
random_lefthand_x['LeftHandPos_x'] = random_lefthand_x['LeftHandPos_x'].sample(frac=1).values
random_lefthand_x = np.asarray(random_lefthand_x)
random_lefthand_y = all_weeks_test.copy(deep=True)
random_lefthand_y['LeftHandPos_y'] = random_lefthand_y['LeftHandPos_y'].sample(frac=1).values
random_lefthand_y = np.asarray(random_lefthand_y)
random_lefthand_z = all_weeks_test.copy(deep=True)
random_lefthand_z['LeftHandPos_z'] = random_lefthand_z['LeftHandPos_z'].sample(frac=1).values
random_lefthand_z = np.asarray(random_lefthand_z)

random_righthand_x = all_weeks_test.copy(deep=True)
random_righthand_x['RightHandPos_x'] = random_righthand_x['RightHandPos_x'].sample(frac=1).values
random_righthand_x = np.asarray(random_righthand_x)
random_righthand_y = all_weeks_test.copy(deep=True)
random_righthand_y['RightHandPos_y'] = random_righthand_y['RightHandPos_y'].sample(frac=1).values
random_righthand_y = np.asarray(random_righthand_y)
random_righthand_z = all_weeks_test.copy(deep=True)
random_righthand_z['RightHandPos_z'] = random_righthand_z['RightHandPos_z'].sample(frac=1).values
random_righthand_z = np.asarray(random_righthand_z)

random_headrot_w = all_weeks_test.copy(deep=True)
random_headrot_w['HeadRot_w'] = random_headrot_w['HeadRot_w'].sample(frac=1).values
random_headrot_w = np.asarray(random_headrot_w)
random_headrot_x = all_weeks_test.copy(deep=True)
random_headrot_x['HeadRot_x'] = random_headrot_x['HeadRot_x'].sample(frac=1).values
random_headrot_x = np.asarray(random_headrot_x)
random_headrot_y = all_weeks_test.copy(deep=True)
random_headrot_y['HeadRot_y'] = random_headrot_y['HeadRot_y'].sample(frac=1).values
random_headrot_y = np.asarray(random_headrot_y)
random_headrot_z = all_weeks_test.copy(deep=True)
random_headrot_z['HeadRot_z'] = random_headrot_z['HeadRot_z'].sample(frac=1).values
random_headrot_z = np.asarray(random_headrot_z)

random_lefthandrot_w = all_weeks_test.copy(deep=True)
random_lefthandrot_w['LeftHandRot_w'] = random_lefthandrot_w['LeftHandRot_w'].sample(frac=1).values
random_lefthandrot_w = np.asarray(random_lefthandrot_w)
random_lefthandrot_x = all_weeks_test.copy(deep=True)
random_lefthandrot_x['LeftHandRot_x'] = random_lefthandrot_x['LeftHandRot_x'].sample(frac=1).values
random_lefthandrot_x = np.asarray(random_lefthandrot_x)
random_lefthandrot_y = all_weeks_test.copy(deep=True)
random_lefthandrot_y['LeftHandRot_y'] = random_lefthandrot_y['LeftHandRot_z'].sample(frac=1).values
random_lefthandrot_y = np.asarray(random_lefthandrot_y)
random_lefthandrot_z = all_weeks_test.copy(deep=True)
random_lefthandrot_z['LeftHandRot_z'] = random_lefthandrot_z['LeftHandRot_z'].sample(frac=1).values
random_lefthandrot_z = np.asarray(random_lefthandrot_z)

random_righthandrot_w = all_weeks_test.copy(deep=True)
random_righthandrot_w['RightHandRot_w'] = random_righthandrot_w['RightHandRot_w'].sample(frac=1).values
random_righthandrot_w = np.asarray(random_righthandrot_w)
random_righthandrot_x = all_weeks_test.copy(deep=True)
random_righthandrot_x['RightHandRot_x'] = random_righthandrot_x['RightHandRot_x'].sample(frac=1).values
random_righthandrot_x = np.asarray(random_righthandrot_x)
random_righthandrot_y = all_weeks_test.copy(deep=True)
random_righthandrot_y['RightHandRot_y'] = random_righthandrot_y['RightHandRot_y'].sample(frac=1).values
random_righthandrot_y = np.asarray(random_righthandrot_y)
random_righthandrot_z = all_weeks_test.copy(deep=True)
random_righthandrot_z['RightHandRot_z'] = random_righthandrot_z['RightHandRot_z'].sample(frac=1).values
random_righthandrot_z = np.asarray(random_righthandrot_z)

In [7]:
tf.reset_default_graph()

x = tf.placeholder("float", [None, 21])
y = tf.placeholder("float", [None, 11])

with tf.Session() as sess:
    model = tf.train.import_meta_graph('models_basic_nn/19_19_02_31/basic_nn_19_19_02_31.meta')
    model.restore(sess, tf.train.latest_checkpoint('models_basic_nn/19_19_02_31'))
    graph = tf.get_default_graph()
        
    W1 = sess.run('W1:0')
    W2 = sess.run('W2:0')
    b1 = sess.run('b1:0')
    b2 = sess.run('b2:0')
            
    layer_1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
    layer_2 = tf.nn.softmax(tf.matmul(layer_1, W2) + b2)
    predictions = tf.equal(tf.argmax(layer_2, 1), tf.argmax(testing_y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions,  tf.float64))
    print("Base accuracy: ", accuracy.eval({x: testing_x, y: testing_y}))
    print('')
    
    print("headpos_x: ", accuracy.eval({x: random_headpos_x, y: testing_y}))
    print("headpos_y: ", accuracy.eval({x: random_headpos_y, y: testing_y}))
    print("headpos_z: ", accuracy.eval({x: random_headpos_z, y: testing_y}))
    print('')
    
    print("lefthandpos_x: ", accuracy.eval({x: random_lefthand_x, y: testing_y}))
    print("lefthandpos_y: ", accuracy.eval({x: random_lefthand_y, y: testing_y}))
    print("lefthandpos_z: ", accuracy.eval({x: random_lefthand_z, y: testing_y}))
    print('')
    
    print("righthandpos_x: ", accuracy.eval({x: random_righthand_x, y: testing_y}))
    print("righthandpos_y: ", accuracy.eval({x: random_righthand_y, y: testing_y}))
    print("righthandpos_z: ", accuracy.eval({x: random_righthand_z, y: testing_y}))
    print('')
    
    print("headrot_w: ", accuracy.eval({x: random_headrot_w, y: testing_y}))
    print("headrot_x: ", accuracy.eval({x: random_headrot_x, y: testing_y}))
    print("headrot_y: ", accuracy.eval({x: random_headrot_y, y: testing_y}))
    print("headrot_z: ", accuracy.eval({x: random_headrot_z, y: testing_y}))
    print('')
    
    print("lefthandrot_w: ", accuracy.eval({x: random_lefthandrot_w, y: testing_y}))
    print("lefthandrot_x: ", accuracy.eval({x: random_lefthandrot_x, y: testing_y}))
    print("lefthandrot_y: ", accuracy.eval({x: random_lefthandrot_y, y: testing_y}))
    print("lefthandrot_z: ", accuracy.eval({x: random_lefthandrot_z, y: testing_y}))
    print('')
    
    print("righthandrot_w: ", accuracy.eval({x: random_righthandrot_w, y: testing_y}))
    print("righthandrot_x: ", accuracy.eval({x: random_righthandrot_x, y: testing_y}))
    print("righthandrot_y: ", accuracy.eval({x: random_righthandrot_y, y: testing_y}))
    print("righthandrot_z: ", accuracy.eval({x: random_righthandrot_z, y: testing_y}))

INFO:tensorflow:Restoring parameters from models_basic_nn/19_19_02_31/basic_nn_19_19_02_31
Base accuracy:  0.31811221104882725

headpos_x:  0.2464755950866065
headpos_y:  0.22531285143445073
headpos_z:  0.16324395342412715

lefthandpos_x:  0.18484717971474127
lefthandpos_y:  0.28512060523402133
lefthandpos_z:  0.2372787526401768

righthandpos_x:  0.21340241841282687
righthandpos_y:  0.2962446951751381
righthandpos_z:  0.22129738995003512

headrot_w:  0.3186984336219854
headrot_x:  0.31773045942883
headrot_y:  0.31686907115806695
headrot_z:  0.31822205980557855

lefthandrot_w:  0.3220645910689698
lefthandrot_x:  0.3165177726587532
lefthandrot_y:  0.310811075364948
lefthandrot_z:  0.3137519767338158

righthandrot_w:  0.3171072580662703
righthandrot_x:  0.3198186734185586
righthandrot_y:  0.31612296970132014
righthandrot_z:  0.31664067275294033


In [ ]:
import matplotlib.pyplot as plt

plt.bar('headpos_x', 0.2464755950866065)
plt.bar('headpos_y', 0.22531285143445073)
plt.bar('headpos_z', 0.16324395342412715)

plt.bar('lefthandpos_x', 0.18484717971474127)
plt.bar('lefthandpos_y', 0.28512060523402133)
plt.bar('lefthandpos_z', 0.2372787526401768)

plt.bar('righthandpos_x', 0.21340241841282687)
plt.bar('righthandpos_y', 0.2962446951751381)
plt.bar('righthandpos_z', 0.22129738995003512)

plt.bar('headrot_w', 0.3186984336219854)
plt.bar('headrot_x', 0.31773045942883)
plt.bar('headrot_y', 0.31686907115806695)
plt.bar('headrot_z', 0.31822205980557855)

plt.bar('lefthandrot_w', 0.3220645910689698)
plt.bar('lefthandrot_x', 0.3165177726587532)
plt.bar('lefthandrot_y', 0.310811075364948)
plt.bar('lefthandrot_z', 0.31822205980557855)

plt.bar('righthandrot_w', 0.3171072580662703)
plt.bar('righthandrot_x', 0.3198186734185586)
plt.bar('righthandrot_y', 0.31612296970132014)
plt.bar('righthandrot_z', 0.31664067275294033)

plt.xlabel('Feature')
plt.xticks(rotation = 90)
plt.ylabel('Testing Accuracy')
plt.ylim([0.0942, 0.0952])
plt.show()

In [ ]:
recordings_test = ['\'week7_3\'', '\'week8_3\'']
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']

week7 = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName = 'week7_3'
    """.format(",".join(students), ",".join(recordings_test)), con)

week8 = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName = 'week8_3'
    """.format(",".join(students), ",".join(recordings_test)), con)

week7_output = []
week8_output = []
def create_labels(row, week):
    output = np.zeros(11)
    output[playerIdx[row['PlayerName']]] = 1
    
    if week == 7:
        week7_output.append(output)
    else:
        week8_output.append(output)
    
week7.apply(lambda x: create_labels(x, 7), axis=1)
week8.apply(lambda x: create_labels(x, 8), axis=1)

week7_y = np.asarray(week7_output)
week8_y = np.asarray(week8_output)

week7 = week7.drop(columns=['PlayerName'])
week8 = week8.drop(columns=['PlayerName'])

week7_x = np.asarray(week7)
week8_x = np.asarray(week8)

print(week7_x.shape, week7_y.shape)
print(week8_x.shape, week8_y.shape)

In [ ]:
tf.reset_default_graph()

labels_7 = [[0 for _ in range(11)] for _ in range(11)]
labels_8 = [[0 for _ in range(11)] for _ in range(11)]

x = tf.placeholder("float", [None, 21])
y = tf.placeholder("float", [None, 11])

with tf.Session() as sess:
    model = tf.train.import_meta_graph('models/19_19_02_31/basic_nn_19_19_02_31.meta')
    model.restore(sess, tf.train.latest_checkpoint('models/19_19_02_31'))
    graph = tf.get_default_graph()
        
    W1 = sess.run('W1:0')
    W2 = sess.run('W2:0')
    b1 = sess.run('b1:0')
    b2 = sess.run('b2:0')
            
    layer_1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
    layer_2 = tf.nn.softmax(tf.matmul(layer_1, W2) + b2)
    
    for i in range(week7_x.shape[0]):
        prediction = layer_2.eval({x: [week7_x[i]]})
        truth = np.argmax(week7_y[i])
        idx = np.argmax(prediction)
        
        labels_7[truth][idx] += 1
        
    for i in range(week8_x.shape[0]):
        prediction = layer_2.eval({x: [week8_x[i]]})
        truth = np.argmax(week8_y[i])
        idx = np.argmax(prediction)
        
        labels_8[truth][idx] += 1

In [ ]:
import matplotlib.pyplot as plt

students = ['Alonzo', 'Dax', 'Emily', 'Isabelle', 'Jake', 'Phillip', 'Rishab', 'Sam', 'justin', 'Alex', 'Jordan']

for i in range(11):
    plt.bar(students, labels_7[i])
    plt.xticks(rotation=90)
    plt.title(students[i])
    plt.show()

In [ ]:
for i in range(11):
    plt.bar(students, labels_8[i])
    plt.xticks(rotation=90)
    plt.title(students[i])
    plt.show()